In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.yaml"
country = "ML"

In [ ]:
from bokeh.io import output_notebook
from IPython.display import Markdown
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Country Foo

## Selection signals

{term}`Selection signal`s found in {term}`cohort`s within this country are shown in the figures below.

In [ ]:
for contig in setup.contigs:
    display(Markdown(f"### Chromosome {contig}"))
    df_signals = page_utils.load_signals(
        contig=contig, query=f"country_alpha2 == '{country}'"
    )
    if len(df_signals) > 0:
        page_utils.plot_signals(df=df_signals, contig=contig)
    else:
        display(Markdown("No signals found."))

## Selection scans

The map below shows geographical regions where mosquito genomic data are available and {term}`genome-wide selection scan`s have been performed.

Click on a map marker to see more information about mosquito {term}`cohort`s for which results are available.

In [ ]:
gdf_cohorts_country = page_utils.gdf_cohorts.query(f"country_alpha2 == '{country}'")

In [ ]:
page_utils.plot_cohorts_map(
    gdf_cohorts=gdf_cohorts_country,
    zoom=6,
    url_prefix="../",
)

In [ ]:
def style_cohorts_table(gdf_cohorts):
    gdf_cohorts = gdf_cohorts.reset_index()
    gdf_cohorts["url"] = "../cohort/" + gdf_cohorts["cohort_id"] + ".html"

    def make_clickable(x):
        url = x["url"]
        name = x["cohort_label"]
        return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(
            url, name
        )

    gdf_cohorts["cohort_label"] = gdf_cohorts.apply(make_clickable, axis=1)

    gdf_cohorts = gdf_cohorts[
        [
            "cohort_label",
            "admin1_name",
            "admin2_name",
            "taxon",
            "year",
            "quarter",
            "cohort_size",
        ]
    ]

    gdf_cohorts = gdf_cohorts.rename(
        columns={
            "cohort_label": "Cohort",
        }
    ).set_index("Cohort")

    gdf_cohorts.columns = [
        "Region",
        "District",
        "Taxon",
        "Year",
        "Quarter",
        "Sample Size",
    ]

    return gdf_cohorts.style


style_cohorts_table(gdf_cohorts_country)